## PROJECT 5

MAP EDA Per Capita using population data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#Folium for Map 
from folium import Map
from folium.plugins import HeatMap



In [ ]:
## Readining in data

# get accident data for all years
df_accident = pd.DataFrame()

for i in range( 2010, 2021):   
    folder_name =  str(i) + '_fars/'
    print(i, folder_name)
    df_accident = df_accident.append(pd.read_csv("./data/" + folder_name + "accident.CSV", encoding='latin-1'))

# get person data for all years
df_person = pd.DataFrame()

for i in range( 2010, 2021):   
    folder_name =  str(i) + '_fars/'
    print(i, folder_name)
    df_person = df_person.append(pd.read_csv("./data/" + folder_name + "person.CSV", encoding='latin-1'))

    
# Make column names lowercase
df_accident.columns = df_accident.columns.str.lower()
df_person.columns = df_person.columns.str.lower()


In [ ]:
##merging person 

df_accident.columns.intersection(df_person.columns)



In [ ]:
#combining accident and person 2010-2020
df_full= pd.merge(df_accident, df_person, on=['state', 'st_case', 've_forms', 'county'], how='inner')

In [ ]:
df_full = df_full.drop(columns='statename_y')

In [ ]:
df_full['state'] = df_full['statename_x']


In [ ]:
# (df_full['state'] == 'Texas').value_counts()
county_pop['state'] = county_pop['state'].str.lstrip()

In [ ]:
(county_pop['state'] == 'Texas').value_counts()

In [ ]:
(df_full['state'] == 'Texas').value_counts()

In [ ]:
df_full[['state', 'countyname','st_case']]

In [ ]:
df_full['countyname']
df_full['countyname'] = df_full['countyname'].str.replace('(\d+)', '')
df_full['countyname'] = df_full['countyname'].str.replace('(','').replace(')','') 
df_full['countyname'] = df_full['countyname'].str.replace(')','') 
df_full['countyname'] = df_full['countyname'].str.rstrip()
df_full['countyname'] = df_full['countyname'].str.lstrip()
df_full['countyname'] = df_full['countyname'].str.lower()
# df_full['countyname'] = df_full['countyname'].str.replace(' county', '')

In [ ]:
df_full['countyname']

In [ ]:
(df_full['countyname'] == 'los angeles').value_counts()

In [ ]:
county_pop['countyname'] = county_pop['countyname'].str.lower()

In [ ]:
df_full = df_full[['year', 'st_case', 'countyname', 'state']].dropna()

In [ ]:
# data from https://www.census.gov/data/datasets/time-series/demo/popest/2020s-counties-total.html

# get population by county data from https://www.census.gov/data/datasets/time-series/demo/popest/2020s-counties-total.html
county_pop1 = pd.read_excel('./data/co-est2019-annres.xlsx', skiprows=3)
county_pop1 = county_pop1.rename(columns={'Unnamed: 0':'county_name'}, errors='ignore').dropna()

county_pop2 = pd.read_excel('./data/co-est2021-pop.xlsx', skiprows=3)

county_pop2 = county_pop2.rename(columns={'Unnamed: 0':'county_name', 'Unnamed: 1':'Estimates Base'}, errors='ignore').dropna()


# merge 2020-2021 census data with 2010-2019 census data
county_pop = pd.merge(county_pop1, county_pop2, on='county_name')

# # split county_name column into county and state columns
county_pop['countyname'] = county_pop['county_name'].str.split(',', expand=True).rename(columns={0:'countyname', 1:'state'})['countyname'].map(lambda x: x.replace('.', ''))
county_pop['state'] = county_pop['county_name'].str.split(',', expand=True).rename(columns={0:'countyname', 1:'state'})['state']

# reorder columns
county_pop = county_pop.loc[:, ['county_name',
                                'Census',
                                'countyname',
                                'state',
                                2010,
                                2011,
                                2012,
                                2013,
                                2014,
                                2015,
                                2016,
                                2017,
                                2018,
                                2019,
                                2020,
                                2021]]

In [ ]:
county_pop = county_pop.drop(index=0, errors='ignore').drop(columns=['county_name', 'Census'], errors='ignore')
county_pop['countyname'] = county_pop['countyname'].str.replace(' county', '')
county_pop.head()

In [ ]:
pop_acc = pd.merge(df_full, county_pop, on=['countyname', 'state'])

In [ ]:
(county_pop['countyname']).value_counts()

In [ ]:
pop_acc['countyname'].value_counts()

In [ ]:
## download latitude, longitude, zip code data from https://www.kaggle.com/datasets/tsnowak/us-geographic-codes
county_latlong_zip = pd.read_csv('./data/us_zip_fips_county.csv', encoding='unicode_escape').rename(columns=lambda x: x.lower().replace(' ', '_'))

In [ ]:
# county_latlong_zip['countyname'] = county_latlong_zip['county_name']
county_latlong_zip['countyname'] = county_latlong_zip['countyname'].str.lower()
county_latlong_zip['countyname'] = county_latlong_zip['countyname'].str.replace(' county', '')
county_latlong_zip['countyname'] = county_latlong_zip['countyname'].str.replace(' borough', '')

In [ ]:
county_latlong_zip = county_latlong_zip.drop(columns='county_name')

In [ ]:
county_latlong_zip['latitude'] = county_latlong_zip['centroid_lat']
county_latlong_zip['longitud'] = county_latlong_zip['centroid_long']
# county_latlong_zip.drop(columns=['centroid_lat', 'centroid_long'])

In [ ]:
county_latlong_zip['countyname'].value_counts()

In [ ]:
county_latlong_zip

In [ ]:
county_latlong_zip.columns

In [ ]:
county_latlong_zip = county_latlong_zip.drop(columns=['zip_code', 'state_abrv', 'state_code', 'ansi_code', 'centroid_lat', 'centroid_long'])

In [ ]:
county_latlong_zip['state'] = county_latlong_zip['state_name']

In [ ]:
county_latlong_zip.columns.intersection(pop_acc.columns)

In [ ]:
map_data = pd.merge(county_latlong_zip, pop_acc, on=['countyname', 'state'])

In [ ]:
map_data.columns

In [ ]:
map_data[['state_name', 'county_code',   'fips_code',  'countyname',
          'latitude','longitud' ,'year','st_case', 2020]]

In [ ]:
map_data_2020 = map_data.loc[map_data['year'] == 2020]

In [ ]:
pd.DataFrame(map_data_2020[['county_code','countyname']].value_counts())

In [ ]:
# (map_data_2020[['county_code','countyname']].value_counts())

In [ ]:
map_data_2020[['county_code','countyname']].value_counts().to_numpy().shape

In [ ]:
# x = pd.DataFrame(data=map_data_2020[['county_code','countyname']].value_counts().to_numpy(), columns='accident_count', index=map_data_2020[['county_code','countyname']].value_counts().index )

In [ ]:
map_data_2020[2020]

In [ ]:
for_map = Map(location=[30.169621, -96.683617], zoom_start=6, control_scale=True)

In [ ]:

hm_wide = HeatMap(list(zip(map_data_2020['latitude'],map_data_2020['longitud'])),
    min_opacity=0.2,
    radius=8, 
    blur=1, 
    max_zoom=1,
)

In [ ]:
## Printing map
# for_map.add_child(hm_wide)

In [ ]:
## Source on folium map https://livecodestream.dev/post/how-to-plot-your-data-on-maps-using-python-and-folium/